In [1]:
from gensim.models import Word2Vec
import pandas as pd
import itertools

In [2]:
all_sentences_df = pd.read_csv("filtered_sentences.csv")
all_sentences_df.head()
all_sentences = all_sentences_df["Sentence"].values

In [3]:
sentences = list(map(lambda x: x.split(),all_sentences[:100_000]))

In [4]:
words = list(itertools.chain(*sentences))

In [5]:
LENGTH_WORD = 6
EMB_SIZE = 100

In [57]:
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
import numpy as np

In [7]:
 mdl = Word2Vec.load('whole_data.model')

In [8]:
pretrained_weights = mdl.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')

def word2idx(word):
    return mdl.wv.get_index(word)
def idx2word(idx):
    return mdl.wv.index_to_key[idx]

Result embedding shape: (535967, 100)
Checking similar words:


In [9]:
len(words)

1529429

In [142]:
train_x = np.zeros([len(words), 5], dtype=np.int32)
train_y = np.zeros([len(words)], dtype=np.int32)
for j in range(len(words) - 5):
    if j % 1_000_000 ==0:
        print(j)
    for i in range(j,j+ 5):
        try:
            train_x[j, i-j] = word2idx(words[i])
        except:
            pass
    try:
        train_y[j] = word2idx(words[j + LENGTH_WORD])
    except:
        pass

0
1000000


In [11]:
train_x.shape

(1529429, 5)

In [178]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/', 
                             verbose=1,
                             save_freq="epoch",)

In [12]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights],trainable=False))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

2022-07-29 13:43:58.072249: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-29 13:43:58.072301: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dsvm): /proc/driver/nvidia/version does not exist
2022-07-29 13:43:58.072952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         53596700  
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 535967)            54132667  
                                                                 
 activation (Activation)     (None, 535967)            0         
                                                                 
Total params: 107,809,767
Trainable params: 54,213,067
Non-trainable params: 53,596,700
_________________________________________________________________


In [15]:
history = model.fit(train_x, train_y, validation_split=0.2, batch_size=256, epochs=1, shuffle=True,callbacks = [checkpoint]).history

4780/4780 [==============================] - 8645s 2s/step - loss: 9.1965 - accuracy: 0.0741 - val_loss: 9.1150 - val_accuracy: 0.0881


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         53596700  
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 535967)            54132667  
                                                                 
 activation (Activation)     (None, 535967)            0         
                                                                 
Total params: 107,809,767
Trainable params: 54,213,067
Non-trainable params: 53,596,700
_________________________________________________________________


In [17]:
model.save('model.h5')

In [176]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10):
    word_idxs = [word2idx(word) for word in text.lower().split()]
    for i in range(num_generated):
        prediction = model.predict(x=np.array(word_idxs).reshape(-1, 1))
        idx = sample(prediction[-1], temperature=0)
        word_idxs.append(idx)
    return ' '.join(idx2word(idx) for idx in word_idxs)

In [29]:
generate_next('მიხეილ',10)

1/1 [==============================] - 0s 37ms/step


'მიხეილ ნაშრომი და და და და და და და და და'

In [177]:
generate_next('მიხეილ',10)

1/1 [==============================] - 0s 72ms/step


'მიხეილ და და და და და და და და და და'